#Notebook for Inference using BIKE


#Installing Libraries

In [1]:

!pip install torchnet
!pip install ftfy
!pip install dotmap
!pip install decord

DEPRECATION: Loading egg at /sfs/gpfs/tardis/home/tkg5kq/.conda/envs/video/lib/python3.11/site-packages/MultiScaleDeformableAttention-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /sfs/gpfs/tardis/home/tkg5kq/.conda/envs/video/lib/python3.11/site-packages/cauchy_mult-0.0.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /sfs/gpfs/tardis/home/tkg5kq/.conda/envs/video/lib/python3.11/site-packages/MultiScaleDeformableAttention-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12

#Downloading Model

In [2]:
import gdown

# The shared link for the second file
file_url = 'https://drive.google.com/uc?id=1tALbViwYTKYShThXp8x8n8upXkDajqre'

# Local path where the file will be saved
output_path = 'ucf-vit-l-f16.pt'  # Replace 'your_second_file_name.ext' with your desired file name and extension

# Download the file
gdown.download(file_url, output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1tALbViwYTKYShThXp8x8n8upXkDajqre
From (redirected): https://drive.google.com/uc?id=1tALbViwYTKYShThXp8x8n8upXkDajqre&confirm=t&uuid=57492af5-1c3d-4ec5-961f-61336f03cf73
To: /content/ucf-vit-l-f16.pt
100%|██████████| 4.65G/4.65G [00:28<00:00, 165MB/s]


'ucf-vit-l-f16.pt'

#Cloning Repo

In [3]:
!git clone https://github.com/hamzakhalil798/BIKE.git

Cloning into 'BIKE'...
remote: Enumerating objects: 358, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 358 (delta 11), reused 6 (delta 6), pack-reused 339 (from 1)
Receiving objects: 100% (358/358), 9.27 MiB | 11.37 MiB/s, done.
Resolving deltas: 100% (150/150), done.


#Inference for single Video

In [4]:
import os
import cv2
from google.colab import files

# Change to the working directory
%cd /content

# Create the necessary directories
os.makedirs("customVideos", exist_ok=True)
os.makedirs("customVideos/Test", exist_ok=True)
os.makedirs("customVideos-Processed", exist_ok=True)

# Function to clear previous data (frames, annotations, and uploaded videos)
def clear_previous_data(test_folder, output_folder, annotations_file):
    # Remove all files in the test folder (uploaded videos)
    for filename in os.listdir(test_folder):
        file_path = os.path.join(test_folder, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)

    # Remove the annotations file if it exists
    if os.path.exists(annotations_file):
        os.remove(annotations_file)

    # Remove all directories and files inside the output folder
    for folder_name in os.listdir(output_folder):
        folder_path = os.path.join(output_folder, folder_name)
        if os.path.isdir(folder_path):
            for root, dirs, files in os.walk(folder_path, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(folder_path)  # Remove the category folder

# Function to upload files to customVideos/Test folder
def upload_videos_to_test_folder():
    uploaded = files.upload()  # Upload files using Colab's file upload widget
    for filename in uploaded.keys():
        # Remove spaces from the filename
        new_filename = filename.replace(" ", "_")
        os.rename(filename, new_filename)
        # Move the uploaded file to the customVideos/Test directory
        os.rename(new_filename, os.path.join("customVideos/Test", new_filename))

# Function to extract frames from videos and save annotations
def extract_frames_from_videos(root_folder, output_folder, annotations_file):
    with open(annotations_file, 'w') as f:
        # Iterate over each category folder in the root directory
        for category in os.listdir(root_folder):
            category_path = os.path.join(root_folder, category)

            if os.path.isdir(category_path):
                # Iterate over each video in the category folder
                for video_file in os.listdir(category_path):
                    video_path = os.path.join(category_path, video_file)

                    # Create the output directory for the video frames
                    output_dir = os.path.join(output_folder, category, os.path.splitext(video_file)[0])
                    os.makedirs(output_dir, exist_ok=True)

                    # Capture the video
                    video_capture = cv2.VideoCapture(video_path)
                    success, frame = video_capture.read()
                    frame_count = 0

                    # Extract frames from the video
                    while success:
                        # Write the frame to the output directory
                        frame_filename = os.path.join(output_dir, f"image_{frame_count:04d}.jpg")
                        cv2.imwrite(frame_filename, frame)

                        # Read the next frame
                        success, frame = video_capture.read()
                        frame_count += 1

                    # Release the video capture object
                    video_capture.release()

                    # Write the video name and frame count to the annotations file
                    videoname = os.path.splitext(video_file)[0]
                    annotation_line = f"Test/{videoname} {frame_count} 0\n"
                    f.write(annotation_line)

# Clear previous data
clear_previous_data("/content/customVideos/Test", "/content/customVideos-Processed", "/content/annotations.txt")

# Call the function to upload files
upload_videos_to_test_folder()

# Extract frames and save annotations
root_folder = "/content/customVideos"  # Root folder containing the videos
output_folder = "/content/customVideos-Processed"  # Output folder for processed images
annotations_file = "/content/annotations.txt"  # File to save the annotations
extract_frames_from_videos(root_folder, output_folder, annotations_file)


/content


Saving v_ApplyEyeMakeup_g01_c01.avi to v_ApplyEyeMakeup_g01_c01.avi


In [5]:
%cd /content/BIKE



/content/BIKE


In [6]:
!python /content/BIKE/inference.py --config /content/BIKE/configs/ucf101/ucf_infer.yaml  --weights /content/ucf-vit-l-f16.pt --dist_url None

Not using distributed mode
100%|████████████████████████████████████████| 890M/890M [00:05<00:00, 178MiB/s]
dropout used:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
dropout used:[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
loading clip pretrained model!
layer=6
video number:1
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warni